# Send value to the Marlowe payout address

Here is a simple contract for depositing value (ada or tokens) into a Marlowe contract that immediately pays them to a role. The payout can be redeemed using Marlowe Runtime's "withdraw" capability.

![Contract for depositing ada to Marlowe's payout address](contract.png)

## Preliminaries

Set the location of the Marlowe Runtime web server.

In [1]:
MARLOWE_RT_WEBSERVER_URL=http://192.168.0.12:13780

Set the address and sign key of the party sending the funds.

In [2]:
SENDER_ADDR=$(cat sender.testnet.address)
SENDER_SKEY=sender.skey

Set the policy ID for the Marlowe roles currency, and the name of the role.

In [3]:
RECEIVER_POLICY=8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
RECEIVER_ROLE=c.marlowe

Set the amount of the payment, along with the policy ID and token name for the token being paid. In this case ada is being deposited.

In [4]:
ADA=1000000
AMOUNT=$((10 * ADA))

In [5]:
TOKEN_NAME="''"
TOKEN_POLICY="''"

Set the deadline for the deposit to be made into the contract.

In [6]:
DEPOSIT_DEADLINE=$((1000 * `date -d '+ 10 minutes' -u +%s`))

## Create the contract

Construct the request for Marlowe Runtime to build the creation transaction.

In [7]:
yaml2json << EOI > request-1.json
version: v1
contract:
  when:
  - case:
      party:
        address: $SENDER_ADDR
      deposits: $AMOUNT
      of_token:
        currency_symbol: $TOKEN_POLICY
        token_name: $TOKEN_NAME
      into_account:
        role_token: $RECEIVER_ROLE
    then: close
  timeout: $DEPOSIT_DEADLINE
  timeout_continuation: close
minUTxODeposit: $((2 * ADA))
roles: $RECEIVER_POLICY
metadata: {}
tags: {}
EOI
cat request-1.json | jq

{
  "contract": {
    "timeout": 1693594514000,
    "timeout_continuation": "close",
    "when": [
      {
        "case": {
          "deposits": 10000000,
          "into_account": {
            "role_token": "c.marlowe"
          },
          "of_token": {
            "currency_symbol": "",
            "token_name": ""
          },
          "party": {
            "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
          }
        },
        "then": "close"
      }
    ]
  },
  "metadata": {},
  "minUTxODeposit": 2000000,
  "roles": "8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d",
  "tags": {},
  "version": "v1"
}


Call the Marlowe Runtime endpoint to build the creation transaction.

In [8]:
curl "$MARLOWE_RT_WEBSERVER_URL/contracts" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SENDER_ADDR" \
  -d @request-1.json \
  -o response-1.json \
  -sS
jq '.resource.txBody' response-1.json > tx-1.unsigned
CONTRACT_ID="$(jq -r '.resource.contractId' response-1.json)"
echo "CONTRACT_ID = $CONTRACT_ID"
CONTRACT_URL="$MARLOWE_RT_WEBSERVER_URL/$(jq -r '.links.contract' response-1.json)"
echo "CONTRACT_URL = $CONTRACT_URL"

CONTRACT_ID = 2138b629a6132dbe96f5b0cbe7c66f4dabbf65de47c88d65d1372cd397bc3ca2#1
CONTRACT_URL = http://192.168.0.12:13780/contracts/2138b629a6132dbe96f5b0cbe7c66f4dabbf65de47c88d65d1372cd397bc3ca2%231


Sign the transaction.

In [9]:
jq '.resource.txBody' response-1.json > tx-1.unsigned
cardano-cli transaction sign \
  --testnet-magic 1 \
  --tx-body-file tx-1.unsigned \
  --out-file tx-1.signed \
  --signing-key-file $SENDER_SKEY

Submit the transaction via Marlowe Runtime.

In [10]:
curl "$CONTRACT_URL" \
  -X PUT \
  -H 'Content-Type: application/json' \
  -d @tx-1.signed \
  -sS

Wait for the transaction to be confirmed.

In [11]:
while [ $(curl -sS "$CONTRACT_URL" | jq .resource.status) != '"confirmed"' ]
do
  echo "$(date -u +"%FT%TZ") Transaction not confirmed."
  sleep 10s
done
echo "$(date -u +"%FT%TZ") Transaction confirmed."

2023-09-01T18:45:19Z Transaction not confirmed.
2023-09-01T18:45:29Z Transaction not confirmed.
2023-09-01T18:45:40Z Transaction not confirmed.
2023-09-01T18:45:50Z Transaction not confirmed.
2023-09-01T18:46:00Z Transaction not confirmed.
2023-09-01T18:46:10Z Transaction not confirmed.
2023-09-01T18:46:20Z Transaction not confirmed.
2023-09-01T18:46:30Z Transaction not confirmed.
2023-09-01T18:46:40Z Transaction not confirmed.
2023-09-01T18:46:50Z Transaction not confirmed.
2023-09-01T18:47:01Z Transaction confirmed.


## Deposit and pay the funds

Construct the request for Marlowe Runtime to deposit the funds.

In [12]:
yaml2json << EOI > request-2.json
version: v1
inputs:
- input_from_party:
    address: $SENDER_ADDR
  into_account:
    role_token: $RECEIVER_ROLE
  of_token:
    currency_symbol: $TOKEN_POLICY
    token_name: $TOKEN_NAME
  that_deposits: $AMOUNT
metadata: {}
tags: {}
EOI
cat request-2.json | jq

{
  "inputs": [
    {
      "input_from_party": {
        "address": "addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j"
      },
      "into_account": {
        "role_token": "c.marlowe"
      },
      "of_token": {
        "currency_symbol": "",
        "token_name": ""
      },
      "that_deposits": 10000000
    }
  ],
  "metadata": {},
  "tags": {},
  "version": "v1"
}


Call the Marlowe Runtime endpoint to build the deposit transaction.

In [13]:
curl "$CONTRACT_URL/transactions" \
  -X POST \
  -H 'Content-Type: application/json' \
  -H "X-Change-Address: $SENDER_ADDR" \
  -d @request-2.json \
  -o response-2.json \
  -sS
TRANSACTION_ID="$(jq -r '.resource.transactionId' response-2.json)"
echo "TRANSACTION_ID = $TRANSACTION_ID"
TRANSACTION_URL="$MARLOWE_RT_WEBSERVER_URL/$(jq -r '.links.transaction' response-2.json)"
echo "TRANSACTION_URL = $TRANSACTION_URL"

TRANSACTION_ID = 5e99b0ce92c57abaaf6e1cd99cc84994ec8c60402add28925cc3e5baf070ce5b
TRANSACTION_URL = http://192.168.0.12:13780/contracts/2138b629a6132dbe96f5b0cbe7c66f4dabbf65de47c88d65d1372cd397bc3ca2%231/transactions/5e99b0ce92c57abaaf6e1cd99cc84994ec8c60402add28925cc3e5baf070ce5b


Sign the transaction.

In [14]:
jq '.resource.txBody' response-2.json > tx-2.unsigned
cardano-cli transaction sign \
  --testnet-magic 1 \
  --tx-body-file tx-2.unsigned \
  --out-file tx-2.signed \
  --signing-key-file $SENDER_SKEY

Submit the transaction via Marlowe Runtime.

In [15]:
curl "$TRANSACTION_URL" \
  -X PUT \
  -H 'Content-Type: application/json' \
  -d @tx-2.signed \
  -sS

Wait for the transaction to be confirmed.

In [16]:
while [ $(curl -sS "$TRANSACTION_URL" | jq .resource.status) != '"confirmed"' ]
do
  echo "$(date -u +"%FT%TZ") Transaction not confirmed."
  sleep 10s
done
echo "$(date -u +"%FT%TZ") Transaction confirmed."

2023-09-01T18:47:05Z Transaction not confirmed.
2023-09-01T18:47:15Z Transaction confirmed.
